# 📝 Prerequisites

OpenAI API Key: You'll need an API key from OpenAI. You can get one from the OpenAI Platform.

JSONL Data File: Your training data must be in a JSONL (JSON Lines) format, where each line is a separate JSON object. The data should follow this structure:

```
{"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "What is the capital of France?"}, {"role": "assistant", "content": "The capital of France is Paris."}]}
{"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "What is 2+2?"}, {"role": "assistant", "content": "2+2 equals 4."}]}
```

In [ ]:
# 1. Install and import necessary libraries
!pip install --upgrade openai

In [ ]:
import openai
import os
from google.colab import userdata

In [ ]:
# 2. Set up your OpenAI API key
# Best practice is to use Colab secrets to store your API key.
# In Colab, click the key icon on the left sidebar -> Add a new secret
# Name the secret 'OPENAI_API_KEY' and paste your key as the value.
try:
    api_key = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = api_key
    client = openai.OpenAI()
    print("API key configured successfully.")
except Exception as e:
    print("Error setting up API key. Please ensure it's stored correctly in Colab secrets.")
    print(e)

In [ ]:
# 3. Upload your training data file to OpenAI
# Make sure your 'training_data.jsonl' file is in your Colab environment.
try:
    with open("training_data_formulations.jsonl", "rb") as f:
        training_file = client.files.create(
            file=f,
            purpose="fine-tune"
        )
    print(f"File uploaded successfully with ID: {training_file.id}")
except FileNotFoundError:
    print("Error: 'training_data.jsonl' not found. Please upload your data file to the Colab session.")

In [ ]:
# 4. Create the Fine-Tuning Job
# Note: As of the latest updates, 'gpt-4o-mini' and 'gpt-3.5-turbo' are common models for fine-tuning.
# 'gpt-4' fine-tuning might be in limited beta.
# The user mentioned "gpt-4.1", but the available models might have different names.
# Please check the latest OpenAI documentation for available model names.
try:
    fine_tuning_job = client.fine_tuning.jobs.create(
        training_file=training_file.id,
        model="gpt-4o-mini" # Or "gpt-3.5-turbo". Check OpenAI docs for the latest gpt-4 model name.
    )
    job_id = fine_tuning_job.id
    print(f"Fine-tuning job created successfully with ID: {job_id}")
    print("You can monitor the job status here:", f"https://platform.openai.com/finetune/{job_id}")

except Exception as e:
    print(f"An error occurred while creating the fine-tuning job: {e}")

In [ ]:
# 5. Monitor the Fine-Tuning Job Status (Optional)
# This will periodically check the status. You will also receive an email
# from OpenAI when the job is complete.
import time

if 'job_id' in locals():
    print("\nMonitoring job status... (This may take some time)")
    while True:
        job_status = client.fine_tuning.jobs.retrieve(job_id)
        status = job_status.status
        print(f"Current job status: {status}")

        if status in ["succeeded", "failed", "cancelled"]:
            if status == "succeeded":
                fine_tuned_model_id = job_status.fine_tuned_model
                print("\n✅ Fine-tuning successful!")
                print(f"Your new model ID is: {fine_tuned_model_id}")
            else:
                print(f"\n❌ Fine-tuning {status}. Please check the OpenAI platform for details.")
            break
        time.sleep(60) # Wait for 60 seconds before checking again

In [ ]:
# 6. Use your Fine-Tuned Model
# Once the job succeeds, you can use your new model ID in API calls.
if 'fine_tuned_model_id' in locals() and fine_tuned_model_id:
    print("\n--- Testing the fine-tuned model ---")
    try:
        completion = client.chat.completions.create(
            model=fine_tuned_model_id,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "What is the capital of France?"} # Use a prompt similar to your training data
            ]
        )
        print("Model Response:")
        print(completion.choices[0].message.content)
    except Exception as e:
        print(f"An error occurred while testing the model: {e}")
else:
    print("\nCould not test the model because the fine-tuned model ID is not available.")
    print("Please retrieve it from the OpenAI platform once the job is complete and use it in your API calls.")